# <strong> 1. Método de Newton-Raphson para Sistemas Não-Lineares </strong>

Supondo um sistema não linear:
$$
f(x_1,x_2, \dots, n)=0 \quad \quad i = 1,2...n
$$

Na vizinhaça de cada solução $\xi_i$, assumimos que cada função pode ser expandida em uma série de Taylor (assom como fizemos em diferenças finitas):
$$
f_i (\mathbf{x + \Delta x}) = f_i(\mathbf{x}) + \sum_{j=1}^{n}  \frac{\partial f_i}{\partial x_j} \Delta x_j + O((\Delta x)^2)
$$

A expressão em seu formato matricial pode ser reescrita como:

$$
\mathbf{f_i (x + }\Delta \mathbf{x}) = \mathbf{f(x)} + \mathbf{J} \Delta \mathbf{  x} + O(\Delta \mathbf{ x}^2)
$$
Uma versão compacta da equação de recorrência para o vetor solução:

$$
\mathbf{x}^{k+1}  = \mathbf{x}^{k} - \left[ \mathbf{J(x}^{k} ) \right] \mathbf{f(x}^{k} )
$$

Podemos usar as diferenças finitas central para aproximar o cálculo dos termos da matriz jacobiana:

- forward $\frac{\partial f_i}{\partial x_j} = \frac{f_i (\dots, x_j, \dots) - f_i(\dots, x_j-h, \dots)}{h} + O(h)$
- backward $\frac{\partial f_i}{\partial x_j} = \frac{f_i (\dots, x_j + h, \dots) - f_i(\dots, x_j, \dots)}{h} + O(h)$

O central portanto:

$$
J_{ij} \equiv \frac{\partial f_i}{\partial x_j} = \frac{f_i (\dots, x_j+h, \dots) - f_i(\dots, x_j-h, \dots)}{2h} + O(h^2)
$$

In [2]:
import numpy as np

# Definindo as funções do sistema de equações
def f1(x):
    return x[0]**2 + x[1]**2 + x[2]**2 - 1

def f2(x):
    return x[0] + x[1] - x[2]

def f3(x):
    return np.exp(x[0]) + x[1]*x[2] - 1

# Função que retorna o sistema de equações como vetor
def system_of_equations(x):
    return np.array([f1(x), f2(x), f3(x)])

# Função para calcular o Jacobiano por diferenças finitas
def jacobian(x, h=1e-4):  # Aumentado o valor de h para maior estabilidade
    n = len(x)  # Número de variáveis (3 neste caso)
    m = 3       # Número de equações
    J = np.zeros((m, n))  # Matriz Jacobiana (m x n)
    
    # Avaliando o sistema no ponto x (função sem perturbação)
    f0 = system_of_equations(x)
    
    # Calculando as derivadas parciais usando diferenças finitas
    for i in range(n):  # Iterar sobre as variáveis (x, y, z)
        # Criar um vetor perturbado para x[i] + h e x[i] - h
        x_forward = np.copy(x)
        x_backward = np.copy(x)
        
        # Perturbações em +h e -h para a variável x[i]
        x_forward[i] += h
        x_backward[i] -= h
        
        # Avaliar o sistema nas duas perturbações
        f_forward = system_of_equations(x_forward)
        f_backward = system_of_equations(x_backward)
        
        # Usar a fórmula da diferença centrada para aproximar as derivadas parciais
        J[:, i] = (f_forward - f_backward) / (2 * h)
    
    return J

# Função para resolver sistemas não lineares usando Newton-Raphson
def newton_raphson_system(x0, tol=1e-6, max_iter=100):
    x = np.copy(x0)
    
    for i in range(max_iter):
        # Avaliar o sistema de equações no ponto atual
        F = system_of_equations(x)
        
        # Calcular o Jacobiano no ponto atual
        J = jacobian(x)
        
        # Verificar se o sistema já está resolvido (erro menor que a tolerância)
        if np.linalg.norm(F, ord=2) < tol:
            print(f"Convergência atingida em {i} iterações.")
            return x
        
        # Resolver o sistema linear J(x) * delta_x = -F(x) usando pseudo-inversa
        delta_x = np.dot(np.linalg.pinv(J), -F)
        
        # Atualizar a solução: x_{k+1} = x_k + delta_x
        x += delta_x
        
        # Exibir o progresso
        print(f"Iteração {i+1}, x = {x}, erro = {np.linalg.norm(F, ord=2)}")
    
    print("Número máximo de iterações atingido.")
    return x

# Ponto inicial (ajustado)
x0 = np.array([0.5, 0.5, 0.5])

# Resolver o sistema usando Newton-Raphson
solucao = newton_raphson_system(x0)

print("Solução encontrada:")
print(solucao)

Iteração 1, x = [-0.39118335  1.26618335  0.875     ], erro = 1.0583949746710124
Iteração 2, x = [-0.40711052  0.9144017   0.50729118], erro = 1.712017688204495
Iteração 3, x = [-0.40699676  0.82322991  0.41623315], erro = 0.2897345688467524
Iteração 4, x = [-0.40699675  0.8165319   0.40953515], erro = 0.018563690168419536
Iteração 5, x = [-0.40699675  0.8164953   0.40949855], erro = 0.00010031739703547426
Convergência atingida em 5 iterações.
Solução encontrada:
[-0.40699675  0.8164953   0.40949855]
